In [1]:
import nltk
import numpy as np
import random
import string # to process standard python strings
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd

import matplotlib as plt
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sklearn.feature_extraction.text import TfidfTransformer

C:\Users\Alex\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Alex\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\Alex\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df=pd.read_csv('Новая выгрузка обр.csv', encoding="cp1251")
df.head(5)

,Автор,Место работы автора,Адресат,Специализация адресата,Содержание,Примечание
0,Антонова С.В.,АО «Интерфакс»,Уразов Р.Н.,Генеральный директор,возможн использован сетев издан информацион ре...,
1,Картошкин С.А.,Министерство просвещения Российской Федерации,Уразов Р.Н.,Генеральный директор,участ совещан вопрос доработк фэ предоставлен ...,
2,Цивилев С.Е.,Администрация Кемеровской области,Уразов Р.Н.,Генеральный директор,командирован саликов к. глушк д. участ заседан...,
3,Волков Г.А.,Министерство транспорта Российской Федерации,Иванюк Л.А.,Административный директор,участ чемпионат мир профессиональн мастерств с...,
4,Атанов И.В.,Ставропольский государственный аграрный универ...,Уразов Р.Н.,Генеральный директор,участ церемон открыт вузовск чемпионат 15 апр ...,


In [4]:
df['Специализация адресата'].value_counts()

Специалист по региональным чемпионатам    1542
Генеральный директор                      1451
Административный директор                  723
Вопросы повышения квалификации             307
Вузовские чемпионаты                       232
Сертификация экспертов WS                  226
Новые компитненции                         129
Вопросы аккредитации                       115
Поздравления                                78
Реклама и сотрудничество                    60
WS Europe                                   17
Участие в выставках                         12
Name: Специализация адресата, dtype: int64

In [11]:
stopwords2 = nltk.corpus.stopwords.words('russian')
stopwords = nltk.corpus.stopwords.words('russian')
stopwords.append('вх')
stopwords.append('об')
stopwords.append('-')
stopwords.append('"')
stopwords.append('``')
stopwords.append(',')
stopwords.append('.')
stopwords.append('»')
stopwords.append('«')
stopwords.append('(')
stopwords.append('\'\'')
stopwords.append(')')

def hello():
    print("Привет, я бот, созданный сортировать письма, напиши мне письмо, и я отправлю его нужному сотруднику!")

def write():
    content=[]
    text = input("Введите краткое содержание вашего письма")
    text = pd.Series(text)
    for i in text:
        content.append(word_tokenize(i, language="russian"))
    new_content=content
    snowball = SnowballStemmer(language="russian")
    for x in range(0,len(new_content)):
        for y in range(0,len(new_content[x])):
            ss = new_content[x][y]
            new_content[x][y]=snowball.stem(ss)
    
    stop_words = stopwords
    filtered_str=[]
    new_content2=[]
    for x in range(0,len(new_content)):
        for y in range(0,len(new_content[x])):
            filtered_str=' '.join((filter(lambda s: s not in stop_words,new_content[x])))
        new_content2 = filtered_str
        
    content = pd.Series(new_content2)
    print(new_content2)
    df=pd.read_csv('Новая выгрузка обр.csv', encoding="cp1251")
    new_row = {'Автор': ' ', 'Место работы автора':' ', 'Адресат':' ', 'Содержание': new_content2, 'Примечание': ' ', 'Специализация адресата': ' '}
    df = df.append(new_row, ignore_index=True)
    X = df['Содержание']
    y = df["Специализация адресата"]

    vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords2)
    X = vectorizer.fit_transform(X).toarray()
    
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(X).toarray()
   
    loaded_model = pickle.load(open('MODEL.sav', 'rb'))
    X_test = X[-1]
    X_test = X_test.reshape(1, -1)
    y_pred = loaded_model.predict(X_test)
    
    return y_pred

def sort():
    x1='1) Специалист по региональным чемпионатам'
    x2='2) Генеральный директор'
    x3='3) Административный директор'
    x4='4) Вопросы повышения квалификации'
    print(x1,x2,x3,x4)
    empl = input("Выберите адресата из списка: ")
    time = input("Выберите период по времени")
    
    df=pd.read_csv('Выгрузка.csv', encoding="cp1251")
    
    df[df['Специализация адресата'].str.contains("Специалист по региональным чемпионатам")]

In [12]:
write()

KeyboardInterrupt: Interrupted by user

In [13]:
sort()

1) Специалист по региональным чемпионатам 2) Генеральный директор 3) Административный директор 4) Вопросы повышения квалификации
Выберите адресата из списка: 2
Выберите период по времени22


ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2
